<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/WeaviateIndex_metadata_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Vespa Vector Store demo


If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-vector-stores-vespa

In [ ]:
!pip install llama-index pyvespa

#### Setting up API key


In [5]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Load documents, build the VectorStoreIndex


In [7]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.vespa import VespaVectorStore
from IPython.display import Markdown, display

## Defining some sample data

Let's insert some documents.


In [8]:
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
            "year": 1994,
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
            "year": 1972,
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
            "theme": "Fiction",
            "year": 2010,
        },
    ),
    TextNode(
        text="To Kill a Mockingbird",
        metadata={
            "author": "Harper Lee",
            "theme": "Mafia",
            "year": 1960,
        },
    ),
    TextNode(
        text="1984",
        metadata={
            "author": "George Orwell",
            "theme": "Totalitarianism",
            "year": 1949,
        },
    ),
    TextNode(
        text="The Great Gatsby",
        metadata={
            "author": "F. Scott Fitzgerald",
            "theme": "The American Dream",
            "year": 1925,
        },
    ),
    TextNode(
        text="Harry Potter and the Sorcerer's Stone",
        metadata={
            "author": "J.K. Rowling",
            "theme": "Fiction",
            "year": 1997,
        },
    ),
]

In [22]:
?VespaVectorStore

Init signature:
VespaVectorStore(
    application_package: vespa.package.ApplicationPackage = ApplicationPackage('hybridsearch', [Schema('doc', Document([Field('id', 'string', ['summary'], None, None, None, None, None, None, None, True, None, None, None, [], None), Field('metadata', 'string', ['summary'], None, None, None, None, None, None, None, True, None, None, None, [], None), Field('text', 'string', ['index', 'summary'], 'enable-bm25', None, None, None, None, True, None, True, None, None, None, [], None), Field('embedding', 'tensor<float>(x[384])', ['input text', 'embed', 'index', 'attribute'], None, None, HNSW('angular', 16, 200), None, None, None, None, False, None, None, None, [], None)], None, None), [FieldSet('default', ['text', 'metadata'])], [RankProfile('bm25', 'bm25sum', None, None, [Function('bm25sum', 'bm25(text)', None)], None, None, None, None, None, None, None, [('query(q)', 'tensor<float>(x[384])')]), RankProfile('semantic', 'closeness(field, embedding)', None, None

### Initilizing the VespaVectorStore


TODO:

- Explain template used
- Option to define your own application package
- Option to use own embedding model (also outside), but must then change template
- Link to pyvespa documentation


In [18]:
from llama_index.core import StorageContext

vector_store = VespaVectorStore()
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes, storage_context=storage_context)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 15/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 20/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 25/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 30/300 seconds...
Using plain http against endpoin

## As retriever


In [19]:
retriever = index.as_retriever()
retriever.retrieve("Who directed inception?")

INFO:llama_index.vector_stores.vespa.base:Query: VectorStoreQuery(query_embedding=None, similarity_top_k=2, doc_ids=None, node_ids=[], query_str='Who directed inception?', output_fields=None, embedding_field=None, mode=<VectorStoreQueryMode.DEFAULT: 'default'>, alpha=None, filters=None, mmr_threshold=None, sparse_top_k=None, hybrid_top_k=None)


Query: VectorStoreQuery(query_embedding=None, similarity_top_k=2, doc_ids=None, node_ids=[], query_str='Who directed inception?', output_fields=None, embedding_field=None, mode=<VectorStoreQueryMode.DEFAULT: 'default'>, alpha=None, filters=None, mmr_threshold=None, sparse_top_k=None, hybrid_top_k=None)


INFO:llama_index.vector_stores.vespa.base:Query body:
 {'hits': 2, 'ranking.profile': 'bm25', 'query': 'Who directed inception?', 'tracelevel': 9, 'yql': 'select * from sources * where userQuery()'}


Query body:
 {'hits': 2, 'ranking.profile': 'bm25', 'query': 'Who directed inception?', 'tracelevel': 9, 'yql': 'select * from sources * where userQuery()'}


INFO:llama_index.vector_stores.vespa.base:Metadata: {'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010, '_node_content': '{"id_": "3032156b-21a9-4ef8-95c6-9809e71b5e07", "embedding": null, "metadata": {"director": "Christopher Nolan", "theme": "Fiction", "year": 2010}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "Inception", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}', '_node_type': 'TextNode', 'document_id': 'None', 'doc_id': 'None', 'ref_doc_id': 'None'}


Metadata: {'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010, '_node_content': '{"id_": "3032156b-21a9-4ef8-95c6-9809e71b5e07", "embedding": null, "metadata": {"director": "Christopher Nolan", "theme": "Fiction", "year": 2010}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "Inception", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}', '_node_type': 'TextNode', 'document_id': 'None', 'doc_id': 'None', 'ref_doc_id': 'None'}


[NodeWithScore(node=TextNode(id_='3032156b-21a9-4ef8-95c6-9809e71b5e07', embedding=None, metadata={'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=2.3017175961610485)]

### As query engine


In [23]:
query_engine = index.as_query_engine()
query_engine.query("Who directed inception?")

INFO:llama_index.vector_stores.vespa.base:Query: VectorStoreQuery(query_embedding=None, similarity_top_k=2, doc_ids=None, node_ids=[], query_str='Who directed inception?', output_fields=None, embedding_field=None, mode=<VectorStoreQueryMode.DEFAULT: 'default'>, alpha=None, filters=None, mmr_threshold=None, sparse_top_k=None, hybrid_top_k=None)


Query: VectorStoreQuery(query_embedding=None, similarity_top_k=2, doc_ids=None, node_ids=[], query_str='Who directed inception?', output_fields=None, embedding_field=None, mode=<VectorStoreQueryMode.DEFAULT: 'default'>, alpha=None, filters=None, mmr_threshold=None, sparse_top_k=None, hybrid_top_k=None)


INFO:llama_index.vector_stores.vespa.base:Query body:
 {'hits': 2, 'ranking.profile': 'bm25', 'query': 'Who directed inception?', 'tracelevel': 9, 'yql': 'select * from sources * where userQuery()'}


Query body:
 {'hits': 2, 'ranking.profile': 'bm25', 'query': 'Who directed inception?', 'tracelevel': 9, 'yql': 'select * from sources * where userQuery()'}


INFO:llama_index.vector_stores.vespa.base:Metadata: {'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010, '_node_content': '{"id_": "3032156b-21a9-4ef8-95c6-9809e71b5e07", "embedding": null, "metadata": {"director": "Christopher Nolan", "theme": "Fiction", "year": 2010}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "Inception", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}', '_node_type': 'TextNode', 'document_id': 'None', 'doc_id': 'None', 'ref_doc_id': 'None'}


Metadata: {'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010, '_node_content': '{"id_": "3032156b-21a9-4ef8-95c6-9809e71b5e07", "embedding": null, "metadata": {"director": "Christopher Nolan", "theme": "Fiction", "year": 2010}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "Inception", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}', '_node_type': 'TextNode', 'document_id': 'None', 'doc_id': 'None', 'ref_doc_id': 'None'}


Response(response='Christopher Nolan', source_nodes=[NodeWithScore(node=TextNode(id_='3032156b-21a9-4ef8-95c6-9809e71b5e07', embedding=None, metadata={'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=2.3017175961610485)], metadata={'3032156b-21a9-4ef8-95c6-9809e71b5e07': {'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010}})

In [21]:
from llama_index.core.vector_stores import FilterOperator, FilterCondition

filters = MetadataFilters(
    filters=[
        MetadataFilter(key="theme", value="Fiction"),
        MetadataFilter(key="year", value=1997, operator=FilterOperator.GT),
    ],
    condition=FilterCondition.OR,
)

retriever = index.as_retriever(filters=filters)
retriever.retrieve("Harry Potter?")

INFO:llama_index.vector_stores.vespa.base:Query: VectorStoreQuery(query_embedding=None, similarity_top_k=2, doc_ids=None, node_ids=[], query_str='Harry Potter?', output_fields=None, embedding_field=None, mode=<VectorStoreQueryMode.DEFAULT: 'default'>, alpha=None, filters=MetadataFilters(filters=[MetadataFilter(key='theme', value='Fiction', operator=<FilterOperator.EQ: '=='>), MetadataFilter(key='year', value=1997, operator=<FilterOperator.GT: '>'>)], condition=<FilterCondition.OR: 'or'>), mmr_threshold=None, sparse_top_k=None, hybrid_top_k=None)


Query: VectorStoreQuery(query_embedding=None, similarity_top_k=2, doc_ids=None, node_ids=[], query_str='Harry Potter?', output_fields=None, embedding_field=None, mode=<VectorStoreQueryMode.DEFAULT: 'default'>, alpha=None, filters=MetadataFilters(filters=[MetadataFilter(key='theme', value='Fiction', operator=<FilterOperator.EQ: '=='>), MetadataFilter(key='year', value=1997, operator=<FilterOperator.GT: '>'>)], condition=<FilterCondition.OR: 'or'>), mmr_threshold=None, sparse_top_k=None, hybrid_top_k=None)


Filter support not implemented yet. Will be ignored.


INFO:llama_index.vector_stores.vespa.base:Query body:
 {'hits': 2, 'ranking.profile': 'bm25', 'query': 'Harry Potter?', 'tracelevel': 9, 'yql': 'select * from sources * where userQuery()'}


Query body:
 {'hits': 2, 'ranking.profile': 'bm25', 'query': 'Harry Potter?', 'tracelevel': 9, 'yql': 'select * from sources * where userQuery()'}


INFO:llama_index.vector_stores.vespa.base:Metadata: {'author': 'J.K. Rowling', 'theme': 'Fiction', 'year': 1997, '_node_content': '{"id_": "6be7efdc-8c99-4deb-bfbb-46bd6920e69e", "embedding": null, "metadata": {"author": "J.K. Rowling", "theme": "Fiction", "year": 1997}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "Harry Potter and the Sorcerer\'s Stone", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}', '_node_type': 'TextNode', 'document_id': 'None', 'doc_id': 'None', 'ref_doc_id': 'None'}


Metadata: {'author': 'J.K. Rowling', 'theme': 'Fiction', 'year': 1997, '_node_content': '{"id_": "6be7efdc-8c99-4deb-bfbb-46bd6920e69e", "embedding": null, "metadata": {"author": "J.K. Rowling", "theme": "Fiction", "year": 1997}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "Harry Potter and the Sorcerer\'s Stone", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}', '_node_type': 'TextNode', 'document_id': 'None', 'doc_id': 'None', 'ref_doc_id': 'None'}


[NodeWithScore(node=TextNode(id_='6be7efdc-8c99-4deb-bfbb-46bd6920e69e', embedding=None, metadata={'author': 'J.K. Rowling', 'theme': 'Fiction', 'year': 1997}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=2.1663224434456927)]